<a href="https://colab.research.google.com/github/sid0312/DC_GAN/blob/master/DC_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import print_function
import random
import torch
import torchvision
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [0]:
manualSeed = 42
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [0]:
ngpu=1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [0]:
print(device)

cuda:0


In [0]:
batchSize = 64
imageSize = 64

transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:211: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [0]:
dataset = torchvision.datasets.CIFAR10(root='/content/drive/My Drive/Module 3 - GANs/data',download=True,transform = transform)
dataloader = torch.utils.data.DataLoader(dataset,batch_size=batchSize,shuffle =True,num_workers=2)

Files already downloaded and verified


In [0]:
def initialize_weights(net):
    classname = net.__class__.__name__
    if classname.find('Conv') != -1:
        net.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        net.weight.data.normal_(1.0, 0.02)
        net.bias.data.fill_(0)

In [0]:
class Generator(nn.Module):
    
    def __init__(self,ngpu):
        super(Generator,self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
            )
    def forward(self, input):
        output = self.main(input)
        return output

generator_net = Generator(ngpu).to(device)
generator_net.apply(initialize_weights)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [0]:
class Discriminator(nn.Module):
    
    def __init__(self,ngpu):
        super(Discriminator,self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(3,64,4,2,1,bias=False),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(64,128,4,2,1,bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(128,256,4,2,1,bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(256,512,4,2,1,bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(512,1,4,1,0,bias=False),
            nn.Sigmoid()
            )  
    def forward(self,input):
        output = self.main(input)
        return output.view(-1)

discriminator_net = Discriminator(ngpu).to(device)
discriminator_net.apply(initialize_weights)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [0]:
criterion = nn.BCELoss()
opt_D = optim.Adam(discriminator_net.parameters(),lr = 0.0002,betas = (0.5,0.99))
opt_G = optim.Adam(generator_net.parameters(),lr = 0.0002,betas = (0.5,0.99))

In [0]:
num_epochs = 30
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader,0):
       # Step 1: Updating the weights of the discriminator's neural network  
        discriminator_net.zero_grad()
        
        # Training the discriminator with an actual image of the dataset 
        real,_ = data
        real = real.to(device)
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0],device=device))
        output = discriminator_net(input)
        errorD_real = criterion(output,target)
        
        # Training the discriminator with a image generated by the generator
        noise = Variable(torch.randn(input.size()[0],100,1,1,device=device))
        fake = generator_net(noise)
        target = Variable(torch.zeros(input.size()[0],device=device))
        output = discriminator_net(fake.detach())
        errorD_fake = criterion(output,target)
        
        #Backpropogation of total error
        errorD_total = errorD_real + errorD_fake
        errorD_total.backward()
        opt_D.step()
        
        # Step 2: Updating the weights of the generator's neural network  
        generator_net.zero_grad()
        target = Variable(torch.ones(input.size()[0],device=device))
        output = discriminator_net(fake)
        errorG = criterion(output,target)
        errorG.backward()
        opt_G.step()
        
        # Step 3: Printing the losses and saving the real images and generated images
        print('[%d/%d][%d/%d] Discriminator\'s loss: %.4f Generator\'s loss: %.4f'%(epoch,num_epochs,i,len(dataloader),errorD_total.data,errorG.data))        
        if i % 100 == 0:
            vutils.save_image(real,'%s/real_samples.png' % "/content/drive/My Drive/Module 3 - GANs/results",normalize = True)
            fake = generator_net(noise)
            vutils.save_image(fake.data,'%s/fake_samples_epoch%03d.png' %("/content/drive/My Drive/Module 3 - GANs/results",epoch),normalize = True)
        

Streaming output truncated to the last 5000 lines.
[23/30][475/782] Discriminator's loss: 0.0150 Generator's loss: 5.3837
[23/30][476/782] Discriminator's loss: 0.0734 Generator's loss: 4.7770
[23/30][477/782] Discriminator's loss: 0.0322 Generator's loss: 5.9638
[23/30][478/782] Discriminator's loss: 0.0182 Generator's loss: 5.3281
[23/30][479/782] Discriminator's loss: 0.0161 Generator's loss: 5.5549
[23/30][480/782] Discriminator's loss: 0.0075 Generator's loss: 6.6475
[23/30][481/782] Discriminator's loss: 0.0126 Generator's loss: 5.5742
[23/30][482/782] Discriminator's loss: 0.0141 Generator's loss: 5.8247
[23/30][483/782] Discriminator's loss: 0.0276 Generator's loss: 4.7812
[23/30][484/782] Discriminator's loss: 0.0071 Generator's loss: 6.5120
[23/30][485/782] Discriminator's loss: 0.0172 Generator's loss: 4.9871
[23/30][486/782] Discriminator's loss: 0.0113 Generator's loss: 6.3793
[23/30][487/782] Discriminator's loss: 0.0277 Generator's loss: 4.8423
[23/30][488/782] Discrimin